In [1]:
import leaderbot as lb
import numpy
import numpy as np
import matplotlib.pyplot as plt
import texplot
import time
import scipy
import pickle

In [2]:
# Load form file instead
filename = '../benchmark/models_train_full.pkl'
#filename = '../benchmark/models_train_split.pkl'
with open(filename, 'rb') as f:
    results = pickle.load(f)

models = results['models']
names = np.array(results['data']['models'])

In [3]:
model = models[23-1]

rnk = model.rank()
scores = model.scores()

scores = scores[rnk]
names = names[rnk]

In [58]:
S = model._get_covariance(centered=False)
S = S[rnk, :][:, rnk]
# D = alg._distance_matrix()

n = S.shape[0]
d = np.array([np.diag(S)]).reshape(n, 1)
e = np.ones((n, 1))
D = d @ e.T + e @ d.T - 2.0 * S

D = np.zeros((n, n), dtype=float)
for i in range(n):
    for j in range(i+1, n):
        D[i, j] = (scores[i] - scores[j]) / np.sqrt(S[i, i] + S[j, j] - 2.0 * S[i, j])
        D[j, i] = D[i, j]

# max_rank = S.shape[0]
# S = S[rnk[:max_rank], :][:, rnk[:max_rank]]
# D = D[rnk[:max_rank], :][:, rnk[:max_rank]]

In [59]:
from sklearn.manifold import MDS
from sklearn.decomposition import KernelPCA

In [46]:
mds = MDS(n_components=3, dissimilarity='precomputed', random_state=42)
points = mds.fit_transform(D)
elev, azim, roll = 8, 115, 0

In [66]:
# Kernel PCA
gamma = 1e-4  # You might need to adjust gamma based on your data
K = np.exp(-gamma * D ** 2)
# K = np.exp(-gamma * D)
m = D.shape[0]
P = np.eye(m, dtype=float) - np.ones((m, m), dtype=float) / m
K = P @ K @ P

kpca = KernelPCA(n_components=3, kernel='precomputed')
points = kpca.fit_transform(K)

# 3D visualization settings
elev, azim, roll = 8, 145, 0

In [67]:
# Plot 3D

n = 40
fontsize = 10

import colorcet as cc

c_map = 'turbo_r'
c_map = 'gist_rainbow'
c_map = 'jet_r'
c_map = cc.cm.CET_R2_r
c_map = cc.cm.CET_R4_r
#c_map = 'nipy_spectral_r'
#c_map = 'gnuplot2_r'

sizes = [5000 * (scores[i] - scores[n]) + 100 * scores[n] for i in range(n)]
colors = np.linspace(0, 1, n)

%matplotlib tk
#%matplotlib inline
#%config InlineBackend.print_figure_kwargs = {'bbox_inches':None}
#%matplotlib notebook
with texplot.theme(use_latex=False):

    #fig, ax = plt.subplots(figsize=(10, 10))
    #fig = plt.figure(figsize=(8, 8))
    #ax = fig.add_subplot(projection='3d')
    fig, ax = plt.subplots(figsize=(8, 6.5), subplot_kw={'projection': '3d'},
                       gridspec_kw=dict(top=1.28, bottom=0.06, left=-0.08, right=0.9))
    
    
    ax.set_proj_type('persp', focal_length=0.17)
    
    ax.scatter(points[:n, 0], points[:n, 1], points[:n, 2],
               s=sizes, c=colors,
               cmap=c_map,
               alpha=0.8,
               edgecolor=(0.0, 0.0, 0.0, 0.0),
               #edgecolor=None,
               linewidth=0.6
              )

    
    for i, name in enumerate(names[:n]):
        ax.text(points[i, 0], points[i, 1], points[i, 2],
                name, fontsize=fontsize, ha='center', va='center')
    
    
    
    # Remove tick labels
    #ax.set_xticks([])
    #ax.set_yticks([])
    #ax.set_zticks([])
    
    #ax.grid(False)
    
    ax.set_xlabel(r'$X_1$')
    ax.set_ylabel(r'$X_2$')
    ax.set_zlabel(r'$X_3$')
    
    # Remove axis panes (background of the plot)
    #ax.xaxis.pane.fill = False
    #ax.yaxis.pane.fill = False
    #ax.zaxis.pane.fill = False
    
    ax.xaxis.pane.set_edgecolor('black')  # Set edge color
    ax.yaxis.pane.set_edgecolor('black')
    ax.zaxis.pane.set_edgecolor('black')
    
    ax.xaxis.pane.set_linewidth(1.5)  # Set edge line width
    ax.yaxis.pane.set_linewidth(1.5)
    ax.zaxis.pane.set_linewidth(1.5)
    
    #plt.subplots_adjust(left=0.05, right=0.8, top=0.8, bottom=0.3)
    
    #plt.tight_layout()
    #ax.set_position([-0.2, -0.1, 0.7, 0.9])  # [left, bottom, width, height]
    
    # Ensure equal aspect ratio for the axes
    #ax.set_box_aspect([1, 1, 0.5])
    
    ax.view_init(elev=elev, azim=azim, roll=roll)
    
    #ax.margins(x=0.2, y=0.00, z=0.2)
    
    eps = 0.0006
    x_min = np.min(points[:n, 0]) - eps
    x_max = np.max(points[:n, 0]) + eps
    y_min = np.min(points[:n, 1]) - eps
    y_max = np.max(points[:n, 1]) + eps
    z_min = np.min(points[:n, 2]) - eps
    z_max = np.max(points[:n, 2]) + eps
    
    ax.axes.set_xlim3d(left=x_min, right=x_max)
    ax.axes.set_ylim3d(bottom=y_min, top=y_max)
    ax.axes.set_zlim3d(bottom=z_min, top=z_max)
    
    #fig.set_size_inches(8, 8)
    #plt.subplots_adjust(left=0, right=0.9, top=1, bottom=0.1)
    #plt.subplots_adjust(left=0, right=1, top=1.4, bottom=0)
    
    #ax.set_box_aspect(aspect=(4, 4, 3), zoom=1)
    #plt.subplots_adjust(left=0, right=1, top=1.0, bottom=0.0)
    
    #plt.subplots_adjust(left=0, right=1, top=1, bottom=0)
    
    #plt.show()
    # fig.tight_layout(pad=40)
    
    fig.canvas.draw()

    plt.show()